# 特征抽取

In [1]:
import pickle
import pandas as pda
from pathlib import Path
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import Counter

In [2]:
# 导入分词结果
with open("../data/all_word.pth", "rb") as f1, open("../data/doc_word.pth", "rb") as f2, open("../data/doc_sentences.pth", "rb") as f3:
    all_word = pickle.load(f1)
    doc_word = pickle.load(f2)
    doc_sentences = pickle.load(f3)

## 文本可读性

文本可读性（Readability）。文本的可读性反映了读者理解文本信息的难易程度。在会计和金融领域中，文本可读性较低时，投资者会难以理解文本编辑者所传达的信息，进而会影响到企业信息传递效果以及投资者的投资行为。

依照王艳艳等（2018）提取的公式，计算句子的迷雾(Fog)指数：

$$Fog=0.4*[(\frac{Words}{Sentences})+100 \cdot (\frac{ComplexWords}{Words})]$$

$$其中Words为分词后的单词总数，Sentences为句子总数，ComplexWords为超过三个汉字的单词数.$$

In [3]:
def cal_fog(doc):  # 用于计算文档的可读性
    Words = len(list(chain(*doc)))  # 获得单词的总数量
    Sentences = len(doc)  # 获得句子的总数量
    ComplexWords = len([word for word in list(chain(*doc)) if len(word) > 3])
    return 0.4 * (Words / Sentences + 100 * ComplexWords / Words)

In [4]:
print(f"文章的可读性为{cal_fog(doc_sentences[0])}")
print(f"文章的可读性为{cal_fog(doc_sentences[1])}")
print(f"文章的可读性为{cal_fog(doc_sentences[2])}")

文章的可读性为4.600358764351216
文章的可读性为4.994743020466494
文章的可读性为5.4203009602216765


## 文本情绪

文本情绪（Sentiment）。在针对文本情绪提取方面，主要包括词典法（Dictionary-based Approach）和有监督机器学习方法（Supervised Learning）。

使用词典法，统计句子中出现的正面词语和负面词语的数量，按照公式如下公式来衡量句子的情绪，Tone越大，代表越偏向正面

$$Pos=积极、正面词的个数/文本总词数$$

$$Neg=消极、负面词的个数/文本总词数$$

$$Tone=\frac{Pos-Neg}{Pos+Neg}$$

In [5]:
dic_path = Path("../input/other/sentiment_dic.xlsx")  # 情感词典的放置位置
dic_data = pda.read_excel(dic_path, sheet_name=["年报正面", "年报负面"], header=None)  # 读取正面和负面的词语
pos_word_dic = dic_data["年报正面"]  # 正面词语词典
neg_word_dic = dic_data["年报负面"]  # 负面词语词典

 按照上述公式计算句子的情绪

In [6]:
def cal_tone(sent):
    pos = 0
    neg = 0
    total_len = len(sent)
    for word in sent:
        if word in pos_word_dic.values:
            pos += 1
        if word in neg_word_dic.values:
            neg += 1
    Pos = pos / total_len
    Neg = neg / total_len
    return (Pos - Neg) / (Pos + Neg)

In [7]:
print(f"该文章的tone值为{cal_tone(doc_word[0])}")

该文章的tone值为0.7615062761506276


## 文本语义关联性

文本语义关联性（Relatedness）。语义关联性就是根据文本中某一类词语去识别文本语义特征的方法。

统计AI相关的词语在文中出现的词频，来判断企业的人工智能指标，这里采用的词典来自于姚加权等（2024）中提出的人工智能词典，词典的前10个词语为：

In [8]:
ai_dic = pda.read_excel("../input/other/人工智能词典v14.xlsx",header=None)
ai_dic.head(10)

,0
0,人工智能
1,计算机视觉
2,图像识别
3,知识图谱
4,智能教育
5,增强现实
6,智能政务
7,特征提取
8,商业智能
9,智能养老


In [9]:
ai_dic_word = ai_dic.iloc[:,0].to_list()
print(f"文章中和AI有关的词语出现的次数为:{[item for item in Counter(doc_word[859]).items() if item[0] in ai_dic_word]}")

文章中和AI有关的词语出现的次数为:[('人工智能', 3), ('商业智能', 1)]


## 文本相似性

在国外，Brown and Tucker（2011）、Hoberg and Phillips（2016）以及Lang and Stice-Lawrence（2015）均运用余弦相似性（Cosine Similarity）来衡量文本的相似程度。

任宏达和王琨（2019）运用该指标计算了不同公司年报中“董事会报告”内容的相似性。通过将所有MD&A中内容分词构建成为词典，然后根据这个词典将每个公司的MD&A部分构建成一个向量，若MD&A中涉及了词典中的第n个词，则这个向量的第n位上即为1，否则为0，然后计算这些向量之间的余弦距离，来衡量两个公司的MD&A部分的相似性。

In [10]:
similarity_word_dic = CountVectorizer(analyzer="word", max_features=2000, binary=True)  # 这里选择将每个MD&A部分构建成一个2000维的向量
vec_res = similarity_word_dic.fit_transform([" ".join(word_list) for word_list in doc_word])  # 构建词典

选取所有上市公司2018年的相关年报，将其转化2000个词表示的向量，然后计算他们的余弦相似度

In [11]:
all_mda_file = Path("../data/md&a")  # 声明放置匹配MD&A内容的文件夹
all_files = list(all_mda_file.glob("*.txt"))  # 获得所有MD&A文件的文件名
MDA_18 = list(all_mda_file.glob("*2018*.txt"))  # 获得爬取的公司中2018年年报的MD&A部分的文件名
MDA_18_index = [all_files.index(fn) for fn in MDA_18]  # 获得2018年年报MD&A部分在all_files中的下标，用于后续从vec_res提取其分词结果
MDA_18_vec = vec_res.toarray()[MDA_18_index]  # 从vec_res提取年份为2018年的MD&A的向量
MDA_18_stkcode = [s.stem.split("_")[-2] for s in MDA_18]

In [ ]:
# 计算两两MD&A之间的相似度，可以使用sklearn提供的cosine_similarity，也可以自己实现
# 自己实现 (MDA_18_vec@MDA_18_vec.T)/(np.linalg.norm(MDA_18_vec,axis=1).reshape((-1,1))@np.linalg.norm(MDA_18_vec,axis=1).reshape((-1,1)).T)
# 使用cosine_similarity
simi_matrix = cosine_similarity(MDA_18_vec)

对于每个公司，取前20个相似度最高的公司的相似度值，累加并除以20，即为该公司的PMC值，该值衡量的是该公司面临的产品市场竞争的激烈程度，计算公式如下：

$$PMC=\frac{\sum similarity\ of\ nearest\ N firms}{N}$$

$$取N为20$$

In [13]:
indexx = simi_matrix.argsort(axis=1)[:, -21:-1]  # 取每个公司与之最相似的20家公司的下标
top_20_simi = simi_matrix[np.arange(simi_matrix.shape[0]).reshape((-1, 1)), indexx]  # 按照下标取出其相似值
all_pmc = top_20_simi.sum(axis=1) / 20  # 计算每个公司的PMC值

In [14]:
pmc_file = pda.DataFrame({"code": MDA_18_stkcode, "pmc": all_pmc})  # 创建DataFrame
pmc_file.to_csv("../data/pmc.csv", index=False)  # 将其写入文件中